In [11]:
import pandas as pd
import scipy as sc
import numpy as np
import sklearn
import pickle
import pathlib as Path
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sb
sb.set(font='IPAexGothic')

import multiprocessing
import itertools
import collections
import datetime
import gc

from tqdm._tqdm_notebook import tqdm

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_columns', 50)

data_path = Path.Path("../data")
result_path = Path.Path("../result")

prefix = 'ana001'

# データ抽出
> ・訓練データ：train_data.csv<br>
> ・テストデータ：test_data.csv<br>
> ※試合情報：game_info

In [12]:
# 訓練データの抽出
train_df = pd.read_csv(data_path / 'train_data.csv', index_col='id')
train_df.head()

,totalPitchingCount,B,S,O,b1,b2,b3,pitcher,pitcherHand,batter,batterHand,gameID,inning,pitchType,speed,ballPositionLabel,ballX,ballY,dir,dist,battingType,isOuts,y
id,,,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,0,False,False,False,今永 昇太,L,ピレラ,R,20202173,1回表,ストレート,149km/h,内角低め,17,J,NaN,NaN,NaN,NaN,0
1,2,1,0,0,False,False,False,今永 昇太,L,ピレラ,R,20202173,1回表,ストレート,149km/h,内角低め,14,I,NaN,NaN,NaN,NaN,1
2,3,1,1,0,False,False,False,今永 昇太,L,ピレラ,R,20202173,1回表,チェンジアップ,137km/h,外角高め,8,D,NaN,NaN,NaN,NaN,0
3,4,2,1,0,False,False,False,今永 昇太,L,ピレラ,R,20202173,1回表,スライダー,138km/h,内角中心,21,G,NaN,NaN,NaN,NaN,2
4,5,2,2,0,False,False,False,今永 昇太,L,ピレラ,R,20202173,1回表,チェンジアップ,136km/h,外角中心,7,F,S,38.3,G,False,4


In [10]:
# テストデータの抽出
test_df = pd.read_csv(data_path / 'test_data.csv', index_col='id')
test_df.head()

,totalPitchingCount,B,S,O,b1,b2,b3,pitcher,pitcherHand,batter,batterHand,gameID,inning
id,,,,,,,,,,,,,
0,2,1,0,0,False,False,False,遠藤 淳志,R,乙坂 智,L,20202564,2回表
1,1,0,0,0,False,False,False,バンデンハーク,R,西川 遥輝,L,20202106,3回裏
2,7,3,2,2,True,False,False,スアレス,R,堂林 翔太,R,20203305,9回裏
3,1,0,0,2,True,False,False,クック,R,井領 雅貴,L,20202650,3回裏
4,2,0,0,2,False,False,False,則本 昂大,R,安達 了一,R,20202339,2回表


In [17]:
# 試合情報の抽出
game_info = pd.read_csv(data_path / 'game_info.csv', index_col=0)
game_info

,startTime,bottomTeam,bgBottom,topTeam,place,startDayTime,bgTop,gameID
0,18:00,DeNA,3,広島,横浜,2020-06-19 18:00:00,6,20202173
1,18:00,ヤクルト,2,中日,神宮,2020-06-19 18:00:00,4,20202174
2,18:00,巨人,1,阪神,東京ドーム,2020-06-19 18:00:00,5,20202175
3,18:00,ソフトバンク,12,ロッテ,PayPayドーム,2020-06-19 18:00:00,9,20202170
4,18:00,オリックス,11,楽天,京セラD大阪,2020-06-19 18:00:00,10,20202171
...,...,...,...,...,...,...,...,...
721,13:00,ソフトバンク,12,ロッテ,PayPayドーム,2020-11-15 13:00:00,9,20203323
722,18:10,巨人,1,ソフトバンク,京セラD大阪,2020-11-21 18:10:00,12,20203326
723,18:10,巨人,1,ソフトバンク,京セラD大阪,2020-11-22 18:10:00,12,20203327
724,18:30,ソフトバンク,12,巨人,PayPayドーム,2020-11-24 18:30:00,1,20203328


# データの保存
> pickle形式で保存<br>

In [19]:
# 訓練データ
train_df.to_pickle(data_path / 'train_df.pkl')
# テストデータ
test_df.to_pickle(data_path / 'test_df.pkl')
# 試合情報
game_info.to_pickle(data_path / 'game_info.pkl')